#  建立对象与导入原始数据

In [1]:
######建立对象
import sys
sys.path.append('E:\\python-dq\\py\\work')
import imp

import dq_work.goods.goods_1 as goods


imp.reload(goods)#再次引入模块，默认只有一次，调试程序时有用

#建立对象
###建立对象的相关参数变量

company='浙江'
#date_Range='202207'
#date_Range_all='20220701-0731'
date_Range='2022'
date_Range_all='2022'
in_file_path='e:/python-dq/work_data/goods_data/in/'
out_file_path='e:/python-dq/work_data/goods_data/out/'
public_path='e:/python-dq/work_data/public_data/out/'

goods_obj=goods.goods(company,date_Range,date_Range_all,in_file_path,out_file_path,public_path)


In [2]:
#导入原始数据
#需要准备的导入文件清单
######################################

in_file1='03_业务报表-销售报表-销售-Partial-data-'+company+ '-'+date_Range_all+'.csv'
in_file2='01-用劵分摊-商品层级-原版-Partial-data-'+company+ '-'+date_Range_all+'.csv'
in_file3='A-分析-门店基础资料-'+company+'.csv'
in_file4='采购收入-'+company+ '-'+date_Range_all+'.csv'
in_file5='B-分析-品类基础资料.csv'
######################################

df_原始销售_Partial,df_原始用券_Partial,df_门店_主档,df_采购收入原始数据,df_类别,df_商品主档= \
goods_obj.load_data(in_file1,in_file2,in_file3,in_file4,in_file5)


导入原始数据 over


# 计算

In [3]:
########计算1########
#合并选择的部分sku的销售数据与券数据
df_sale_coupons=goods_obj.sale_coupons_join(df_原始销售_Partial,df_原始用券_Partial)

单店单品的销售数据与券数据合并完成
销售净额    3.081705e+08
dtype: float64
万家承担金额（去税）    1.748683e+07
dtype: float64


In [4]:
########计算2########
#返回所选择的sku的反馈
df_goods_range=goods_obj.filter_goods_range(df_sale_coupons)


In [7]:
########计算3########
#分摊完整的OI，到大类与门店较粗的层级

import dq_work.whole.whole_1 as whole


imp.reload(whole)#再次引入模块，默认只有一次，调试程序时有用

#建立对象
###建立对象的相关参数变量

#company='浙江'
#date_Range='202207'
#date_Range_all='20220701-0731'
in_file_path='e:/python-dq/work_data/whole_data/in/'
out_file_path='e:/python-dq/work_data/whole_data/out/'
public_path='e:/python-dq/work_data/public_data/out/'

whole_obj=whole.whole(company,date_Range,date_Range_all,in_file_path,out_file_path,public_path)

#导入原始数据
#需要准备的导入文件清单
######################################

in_file1='03_业务报表-销售报表-大类-供应商-门店-'+company+ '-'+date_Range_all+'.csv'
#券是到单品级的原始数据
#in_file2='01-用劵分摊-商品层级-原版-'+company+ '-all-'+date_Range_all+'.csv'
#券是大类层级的原始数据
in_file2='02-用券分摊-大类层级-数据表-'+company+ '-'+date_Range_all+'.csv'

in_file3='A-分析-门店基础资料-'+company+'.csv'
in_file4='采购收入-'+company+ '-'+date_Range_all+'.csv'
in_file5='B-分析-品类基础资料.csv'
######################################

df_原始销售_all,df_原始用券_all,df_门店_主档,df_采购收入原始数据,df_类别= \
whole_obj.load_data(in_file1,in_file2,in_file3,in_file4,in_file5)


#分摊完整的oi

df_oi1,df_oi2=whole_obj.shop_category_oi_share(df_原始销售_all,df_采购收入原始数据,df_类别)
df_oi3=df_oi1.append(df_oi2)



导入原始数据 over
金额    471565756.2
dtype: float64
完整的OI在大类加门店字段上分摊计算完成


In [8]:
########计算2-1########
#增加df_oi3中的、属于3101的门店与渠道的范围
bool_ls1= df_oi3['大类编码']==3101
df_oi_ls=df_oi3[bool_ls1]
df_goods_range2=goods_obj.filter_goods_range(df_oi_ls)
df_goods_range3=df_goods_range.append(df_goods_range2)

df_goods_range3.drop_duplicates(subset = ['大类编码','门店DC编码','渠道'],keep='first',inplace=True)


In [9]:
########计算4########
#分摊OI，到所选择的单品层级

###过滤符合选择sku范围内的OI
import pandas as pd
df_tmp_ceshi=pd.merge(df_goods_range3,df_oi3,on=['大类编码','门店DC编码','渠道'],how='right')
bool_ce= df_tmp_ceshi['pd'].isnull()
df_oi4=df_tmp_ceshi[~bool_ce]

###根据选择的部分范围的销售数据，分摊oi到单店单品
df_oi=goods_obj.partial_shop_sku_oi_share(df_原始销售_Partial,df_oi4,df_类别)

OI分摊金额    23400509.07
dtype: float64
到单店单品的OI 分摊计算完成


In [10]:
import numpy as np
#合并处理销售、券、oi并输出结果df
df_all1=goods_obj.partial_merge_out_df(df_sale_coupons,df_oi)

#精加工输出的DF
df_out=goods_obj.finish_machining_outfile(df_all1,df_商品主档,df_类别,df_门店_主档)

df_out.fillna(0,inplace=True)
df_out = df_out.replace([np.inf, -np.inf],0)




df合并处理finish
精加工 dataframe over


# 导出文件

In [61]:
#输出单店单品的销售df到外面的文件中
goods_obj.put_outfile(df_out)

单店单品的销售数据的excel文件导出 over


C:\Users\daiqiang\anaconda3\lib\site-packages\xlsxwriter\workbook.py:336: UserWarning: Calling close() on already closed file.
  warn("Calling close() on already closed file.")


In [11]:
#将sku的对应供应商编码名称结果落库mysql
#落库的table

in_table='Partial_goods' + date_Range
goods_obj.input_mysql(df_out,in_table)

input mysql


C:\Users\daiqiang\anaconda3\lib\site-packages\pandas\io\sql.py:1444: UserWarning: The provided table name 'Partial_goods2022' is not found exactly as such in the database after writing the table, possibly due to case sensitivity issues. Consider using lower case table names.
  warnings.warn(msg, UserWarning)


# 添加单品的库存周转相关字段

In [41]:
#导入单店单品库存的原始数据
#需要准备的导入文件清单
######################################
in_file1='03_业务报表-销售报表-库存-Partial-data-'+company+ '-'+date_Range_all+'.csv'
######################################

df_原始库存= \
goods_obj.load_data_stock(in_file1)


导入原始库存数据 over
库存金额    1.310402e+10
dtype: float64


In [45]:
########计算########
#计算库存数据
df_stock1=goods_obj.stock_compute(df_原始库存)

#销售df与库存df的合并
###销售df，去上述的精加工后的销售毛利df--df_out
df_sale_stock1=goods_obj.merge_sale_stock(365,df_out,df_stock1)



单店单品库存计算finish
单店单品的销售与库存合并处理finish


In [46]:
#添加库存原始数据中的sku主档信息
df_商品主档_stock=goods_obj.create_sku_master_data_stock(df_商品主档,df_原始库存)
#精加工输出的DF
df_out2=goods_obj.finish_machining_outfile(df_sale_stock1,df_商品主档_stock,df_类别,df_门店_主档)

精加工 dataframe over


In [47]:
#输出单店单品的销售毛利与库存df到外面的文件中
goods_obj.put_outfile_stock(df_out2)

单店单品的销售毛利库存数据的excel文件导出 over


# 下述保留，用于暂无用的历史保留之用

### 1-针对选择了模块编码的部分数据的goods分析

In [3]:
########计算########
#合并销售数据与券数据
df_sale_coupons=goods_obj.sale_coupons_join(df_原始销售,df_原始用券)

#分摊oi
######模块编码的选择
model_bm=24
df_oi1,df_oi2=goods_obj.model_shop_sku_oi_share(df_原始销售,df_采购收入原始数据,df_类别,model_bm)

#合并处理销售、券、oi并输出结果df
df_all1=goods_obj.merge_out_df(df_sale_coupons,df_oi1,df_oi2)

#精加工输出的DF
df_out=goods_obj.finish_machining_outfile(df_all1,df_商品主档,df_类别,df_门店_主档)


单店单品的销售数据与券数据合并完成
销售净额    2.279540e+07
dtype: float64
万家承担金额（去税）    1.627828e+06
dtype: float64
金额    1404808.65
dtype: float64
到单店单品的OI 分摊计算完成
df合并处理finish
精加工 dataframe over


### 0r 3-选择销售渠道的goods的分析

In [76]:
########计算########
#合并销售数据与券数据
df_sale_coupons=goods_obj.sale_coupons_join(df_原始销售,df_原始用券)


#######################以大类完整数据，准备完整的oi分摊的数据######################################
#建立对象
import sys
sys.path.append('E:\\python-dq\\py\\work')
import imp

import dq_work.whole.whole_1 as whole


imp.reload(whole)#再次引入模块，默认只有一次，调试程序时有用

###建立对象的相关参数变量

company='上海'
date_Range='202207'
date_Range_all='20220701-0731'
in_file_path='e:/python-dq/work_data/whole_data/in/'
out_file_path='e:/python-dq/work_data/whole_data/out/'
public_path='e:/python-dq/work_data/public_data/out/'

whole_obj=whole.whole(company,date_Range,date_Range_all,in_file_path,out_file_path,public_path)

#导入原始数据
#需要准备的导入文件清单


in_file1='03_业务报表-销售报表-大类-供应商-门店-'+company+ '-'+date_Range_all+'.csv'
in_file2='01-用劵分摊-商品层级-原版-'+company+ '-all-'+date_Range_all+'.csv'
in_file3='A-分析-门店基础资料-'+company+'.csv'
in_file4='采购收入_'+company+ '_'+date_Range_all+'.csv'
in_file5='B-分析-品类基础资料.csv'


df_原始销售1,df_原始用券1,df_门店_主档1,df_采购收入原始数据1,df_类别1= \
whole_obj.load_data(in_file1,in_file2,in_file3,in_file4,in_file5)

#分摊oi
df_oi1,df_oi2=whole_obj.shop_category_oi_share(df_原始销售1,df_采购收入原始数据1,df_类别1)



df_oi3=df_oi1.append(df_oi2)
#以上df_oi3，是完整数据的oi的分摊结果###########################################


#真正分摊渠道的oi
bool_渠道=df_oi3['渠道'].str.contains('企事业|线上')
df_oi3=df_oi3[bool_渠道]

df_oi=goods_obj.channel_shop_sku_oi_share(df_原始销售,df_oi3,df_类别)


#合并处理销售、券、oi并输出结果df
df_all1=goods_obj.partial_merge_out_df(df_sale_coupons,df_oi)

#精加工输出的DF
df_out=goods_obj.finish_machining_outfile(df_all1,df_商品主档,df_类别,df_门店_主档)






单店单品的销售数据与券数据合并完成
销售净额    3.516691e+07
dtype: float64
万家承担金额（去税）    5.231305e+06
dtype: float64
导入原始数据 over
金额    6154987.87
dtype: float64
OI 分摊计算完成
OI分摊金额    1.485852e+06
dtype: float64
到单店单品的OI 分摊计算完成
df合并处理finish
精加工 dataframe over
